<a href="https://colab.research.google.com/github/benwulfe/FireShooter/blob/master/ddf_ee_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_source_control_pane()
# ddfimport.ddf_import_common()


In [ ]:
import importlib
import eeddf
importlib.reload(eeddf)
import raster
importlib.reload(raster)
import eeraster
importlib.reload(eeraster)
import dataset
importlib.reload(dataset)
import numpy as np
import time

In [ ]:
# Brazil national territory extends 4,395 kilometers (2,731 mi) from north to
# south (5°16'20" N to 33°44'32" S latitude), and 4,319 kilometers (2,684 mi)
# from east to west (34°47'30" W to 73°59'32" W longitude)

def simple_dem_raster_query_latlon():
  value = eeraster.dem().value_at(-50.0, -10.0)
  print(value)
  assert(value > 0)

def simple_dem_raster_query_latlon_int64():
  value = eeraster.dem().value_at(-50, -10)
  print(value)
  assert(value > 0)

def water_dem_raster_query_latlon():
  value = eeraster.dem().value_at(-40.0, -20.0)
  print(value)
  assert(np.isnan(value))

def list_dem_raster_query_latlon_int64():
  values = eeraster.dem().values_at([[-50, -10], [-50, -9.5], [-40, -9.5], [-30, -10]])
  print(values)
  assert(len(values) > 0)
  assert(values[0] > 0)
  assert(values[1] > 0)
  assert(values[2] > 0)
  assert(np.isnan(values[3]))

def ee_performance():
  #-8,-35 to -8,-75
  start_time = time.time()

  coords = [(n, -8.0) for n in np.arange(-75.0, -35.0, 0.01)]
  print("sending batch of coords of size, ", len(coords))
  values = eeraster.dem().values_at(coords)
  assert(len(values) == len(coords))

  end_time = time.time()
  total_time = end_time - start_time
  print(f"Total time: {total_time:.2f} seconds")
  print(f"Throughput: {len(coords) / total_time:.2f} qps")

def reference_sample_feature_test():
  ref_samples = dataset.load_reference_samples()
  print(ref_samples.head())
  print("************* Loading DEM ******************")
  df = eeraster.dem().values_at_df(ref_samples, "dem")
  print(df.head())

def performance_xtreme_test():
  eeraster.set_ee_options(parallel_ops=40, chunk_size=120)
  #-8,-35 to -8,-75
  start_time = time.time()

  coords = [(n, -8.0) for n in np.arange(-75.0, -35.0, 0.01)]
  print("sending batch of coords of size, ", len(coords))
  values = eeraster.dem().values_at(coords)
  assert(len(values) == len(coords))

  end_time = time.time()
  total_time = end_time - start_time
  print(f"Total time: {total_time:.2f} seconds")
  print(f"Throughput: {len(coords) / total_time:.2f} qps")
  eeraster.set_ee_options(parallel_ops=30, chunk_size=80)


simple_dem_raster_query_latlon()
simple_dem_raster_query_latlon_int64()
list_dem_raster_query_latlon_int64()
water_dem_raster_query_latlon()
ee_performance()
reference_sample_feature_test()
performance_xtreme_test()


In [ ]:
def test_ingestion():
    isoscape_path = raster.get_raster_path("test_isoscape.tiff")
    raster.stamp_isoscape(isoscape_path, "p_value", str(0.05))

    test_asset = "projects/river-sky-386919/assets/testing/test_isoscape"
    eeraster.ingest_isoscape(isoscape_path, test_asset, allow_overwrite=True)

def test_stamped():
    test_asset = "projects/river-sky-386919/assets/testing/test_isoscape"
    assert eeraster.get_props(test_asset)["p_value"] == str(0.05)

test_ingestion()
test_stamped()